This script do:

- add features
   - mean encode 
       - by (ip,app)
       - by (app,channel)
       - os_mean_target
       - hour_mean_target       
   - base
       - ip_unicnt_channel
       
- features base
    - use only (gain importance)
        - app_mean_target
        - app
        - channel_mean_target
        - next_click_dt
        - channel_nunique_ip
        - os
        - channel
        - hour        
        - app_nunique_channel
        - channel_cnt_ip_app
        - hour_nunique_day_ip
        - app_nunique_ip
        - app_nunique_ip_device_os
        - device_mean_target 	
        - channel_cnt_ip_day_hour
        - app_cumcnt_ip_device_os
        - ip_mean_target        
        - device

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import pytz
%matplotlib inline 
import gc,time

In [3]:
with pd.HDFStore('../input/feat/feat4_trn_day8_val_day9.h5') as store:
    print(store.keys())
    test_df = store['test_df']
    train_df = store['train_df']
    val_df = store['valid_df']

['/test_df', '/train_df', '/valid_df']


In [8]:
trn_df = train_df.append(val_df)
trn_df = trn_df.append(test_df)
del train_df,test_df,val_df;gc.collect()

7

____

In [10]:
trn_df.columns

Index(['app', 'app_cumcnt_ip_device_os', 'app_mean_target',
       'app_nunique_channel', 'app_nunique_ip', 'app_nunique_ip_device_os',
       'channel', 'channel_cnt_ip_app', 'channel_cnt_ip_day_hour',
       'channel_mean_target', 'channel_nunique_ip', 'click_id', 'click_time',
       'day', 'device', 'device_mean_target', 'hour', 'hour_nunique_ip_day',
       'ip', 'ip_app_os_device_mean_target', 'ip_mean_target', 'is_attributed',
       'next_click_dt', 'os'],
      dtype='object')

In [11]:
trn_df.drop(['ip_app_os_device_mean_target'],axis=1,inplace=True)

In [13]:
gc.collect()

print('trn_df :{}'.format(trn_df.shape)) # 
# print('test_df:{}'.format(test_df.shape))
# del test_df

trn_df :(143984060, 23)


- len_trn  : 125193591 

- len_tst  : 18790469

# helper func

In [14]:
def encode_agg_feature(train_df, selcols, groupby, aggregator = 'nunique'):
    usecols = [e for e in selcols if e not in  groupby]    
    
    if aggregator == 'nunique':  
        colname= usecols[-1] + '_nunique_' + '_'.join(groupby)
        if colname in train_df.columns:
            train_df.drop(colname,axis=1,inplace=True)
            
        gp = train_df[selcols].groupby(groupby)[usecols].nunique().reset_index().\
            rename(columns = {
                usecols[-1] : usecols[-1] + '_nunique_' + '_'.join(groupby)
            })
        train_df = train_df.merge(gp, how='left', on=groupby)

    elif aggregator == 'cumcount':
        colname = usecols[-1] + '_cumcnt_' + '_'.join(groupby)
        if colname in train_df.columns:
            train_df.drop(colname,axis=1,inplace=True)
            
        gp = train_df[selcols].groupby(groupby)[usecols].cumcount()        
        train_df[colname] = gp.values
    
    elif aggregator == 'count':
        colname = usecols[-1] + '_cnt_' + '_'.join(groupby)
        if colname in train_df.columns:
            train_df.drop(colname,axis=1,inplace=True)
        gp = train_df[selcols].groupby(groupby)[usecols].count().reset_index().\
            rename(columns = {
                usecols[-1] : colname
            })
        train_df = train_df.merge(gp, how='left', on=groupby)
        
    elif aggregator in ['var','mean']:
        agg=np.var if aggregator == 'var' else np.mean
        
        colname = usecols[-1] + '_'+ str(aggregator) +'_' + '_'.join(groupby)
        
        if colname in train_df.columns:
            train_df.drop(colname,axis=1, inplace=True)
            
        gp = train_df[selcols].groupby(groupby).agg(agg).reset_index().\
            rename(columns = {
                usecols[-1] : colname
            })
        train_df = train_df.merge(gp, how='left', on=groupby)
    
    return train_df.iloc[:,-1]

#### feat_base                     

        - [o] ip_unicnt_channel

In [16]:
print('num of nunique "IP" by "channel"...',end='\t')

ip_unicnt_channel_df = encode_agg_feature(trn_df, groupby=['channel'],selcols=['ip','channel'],aggregator='nunique')
print(ip_unicnt_channel_df.name)
trn_df[ip_unicnt_channel_df.name] = ip_unicnt_channel_df.values.astype('uint16'); del ip_unicnt_channel_df;gc.collect()


num of nunique "IP" by "channel"...	ip_nunique_channel


1219

In [17]:
len_tst = 18790469
len_trn = 143984060 - 18790469

#### mean encode ON TRAINNING ONLY NOT ON TEST ###
- (app,channel)
- (ip, app)
- os
- hour

In [4]:
# test_df = trn_df.iloc[len_trn:,]
# trn_df  = trn_df.iloc[:len_trn,]

In [18]:
debug = 0
if debug:
    len_tst = 10000
    len_trn = 10000
else:
    len_tst = 18790469
    len_trn = 143984060 - 18790469
test_df = trn_df.iloc[len_trn:,]
trn_df = trn_df.iloc[:len_trn,]

In [19]:
print('trn_df shape :',trn_df.shape)
print('test_df shape:',test_df.shape)

trn_df shape : (125193591, 24)
test_df shape: (18790469, 24)


In [20]:
trn_df.columns

Index(['app', 'app_cumcnt_ip_device_os', 'app_mean_target',
       'app_nunique_channel', 'app_nunique_ip', 'app_nunique_ip_device_os',
       'channel', 'channel_cnt_ip_app', 'channel_cnt_ip_day_hour',
       'channel_mean_target', 'channel_nunique_ip', 'click_id', 'click_time',
       'day', 'device', 'device_mean_target', 'hour', 'hour_nunique_ip_day',
       'ip', 'ip_mean_target', 'is_attributed', 'next_click_dt', 'os',
       'ip_nunique_channel'],
      dtype='object')

In [21]:
from sklearn.model_selection import StratifiedKFold
print('copy trn_df to make new features')
train_new = trn_df.copy()

print('create new cols')
cols = [['ip','app'],['app','channel'], 'os','hour']
for col in cols :
    if isinstance(col,list):
        col_str = '_'.join(col)
    else:
        col_str = col
    train_new[col_str + '_mean_target'] = 0

print('StratifiedKFold...')
y_tr = trn_df.is_attributed.values.astype(np.int8) # target 
skf = StratifiedKFold(4, random_state=0)
skf.get_n_splits(X=trn_df,y=y_tr)

gc.collect()
for fold, (tr_idx, val_idx) in enumerate(skf.split(train_new,y_tr)):
    
    ## generate features 
    
    X_tr ,X_val = trn_df.iloc[tr_idx], trn_df.iloc[val_idx]
    
    print('fold:{} mean encoding...'.format(fold),end='\n\t')
#     print('tr_idx:{}'.format(tr_idx))
    for col in cols:        
        print(col,end='\t')
        if isinstance(col,str):
            means = X_val[col].map(X_tr.groupby(col).is_attributed.mean()) ## map mean encoding in X_tr to X_val
            col_str = col
            
        elif isinstance(col,list):
            col_str = '_'.join(col)
            gp = X_tr.groupby(col).is_attributed.mean().reset_index().rename(columns={'is_attributed':col_str+'_mean_target'})
            means = X_val[col].merge(gp,how='left',on=col)[col_str+'_mean_target']
            del gp;
            
        X_val[col_str + '_mean_target'] = means.astype('float16')
        del means;gc.collect()
        
        
        
    train_new.iloc[val_idx] = X_val
    print('')    
    del X_tr,X_val;gc.collect()
            
prior = trn_df.is_attributed.mean()

col_used = [ '_'.join(col) + '_mean_target' if isinstance(col,list) else col for col in cols]
train_new[col_used] = train_new[col_used].fillna(prior)

trn_df = train_new
del train_new;gc.collect()
print('complete')

copy trn_df to make new features
create new cols
StratifiedKFold...
fold:0 mean encoding...
	['ip', 'app']	

C:\Program Files\Anaconda3\envs\py36\lib\site-packages\ipykernel\__main__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['app', 'channel']	os	hour	
fold:1 mean encoding...
	['ip', 'app']	['app', 'channel']	os	hour	
fold:2 mean encoding...
	['ip', 'app']	['app', 'channel']	os	hour	
fold:3 mean encoding...
	['ip', 'app']	['app', 'channel']	os	hour	
complete


In [22]:
trn_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125193591 entries, 59709852 to 184903442
Data columns (total 28 columns):
app                         uint16
app_cumcnt_ip_device_os     uint16
app_mean_target             float16
app_nunique_channel         uint8
app_nunique_ip              uint8
app_nunique_ip_device_os    uint8
channel                     uint16
channel_cnt_ip_app          uint16
channel_cnt_ip_day_hour     uint16
channel_mean_target         float16
channel_nunique_ip          uint8
click_id                    float64
click_time                  datetime64[ns, Asia/Shanghai]
day                         uint8
device                      uint16
device_mean_target          float16
hour                        uint8
hour_nunique_ip_day         uint8
ip                          uint32
ip_mean_target              float16
is_attributed               float64
next_click_dt               uint16
os                          uint16
ip_nunique_channel          uint16
ip_app_mean_ta

___ 

#### mean encode on TEST

In [43]:
cols = [['app','channel'],['ip','app'],'os','hour']

selcols = ['ip','app','os','channel','hour','is_attributed']

for col in cols:
    print('target_mean col:{}...'.format(col))
    gp = trn_df[selcols].groupby(col)
    if isinstance(col,list):
        col_str = '_'.join(col)
        gp = gp.is_attributed.mean().reset_index().rename(columns={'is_attributed':col_str+'_mean_target'})
        means = test_df.merge(gp,how='left',on=col)[col_str+'_mean_target'].astype('float16')
    else:
        col_str = col
        means = test_df[col].map(gp.is_attributed.mean()).astype('float16')
        
    test_df[col_str +'_mean_target'] = means
    del means,gp;gc.collect()
    
prior = trn_df.is_attributed.mean()


col_used = [ '_'.join(col) + '_mean_target' if isinstance(col,list) else col for col in cols]
test_df[col_used].fillna(prior)

gc.collect()
print('complete')

target_mean col:['app', 'channel']...
target_mean col:['ip', 'app']...
target_mean col:os...
target_mean col:hour...
complete


In [44]:
test_df.shape

(18790469, 28)

In [46]:
test_df.head()

,app,app_cumcnt_ip_device_os,app_mean_target,app_nunique_channel,app_nunique_ip,app_nunique_ip_device_os,channel,channel_cnt_ip_app,channel_cnt_ip_day_hour,channel_mean_target,...,ip,ip_mean_target,is_attributed,next_click_dt,os,ip_nunique_channel,app_channel_mean_target,ip_app_mean_target,os_mean_target,hour_mean_target
0,9,3,0.001114,4,31,4,107,104,34,0.000529,...,5744,0.001394,NaN,50000,3,48100,0.001152,0.000000,0.000798,0.00276
1,9,187,0.001114,2,74,22,466,1791,403,0.001122,...,119901,0.001630,NaN,399,3,32606,0.001239,0.000666,0.000798,0.00276
2,21,2408,0.000134,10,61,37,128,592,229,0.000214,...,72287,0.000984,NaN,88,19,21156,0.000101,0.000000,0.001735,0.00276
3,15,1679,0.000223,1,59,37,111,493,239,0.000265,...,78477,0.002548,NaN,425,13,57203,0.000265,0.000000,0.001463,0.00276
4,12,114,0.000107,1,29,22,328,90,60,0.000097,...,123080,0.002213,NaN,1208,13,3214,0.000097,0.000000,0.001463,0.00276


In [45]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18790469 entries, 0 to 18790468
Data columns (total 28 columns):
app                         uint16
app_cumcnt_ip_device_os     uint16
app_mean_target             float16
app_nunique_channel         uint8
app_nunique_ip              uint8
app_nunique_ip_device_os    uint8
channel                     uint16
channel_cnt_ip_app          uint16
channel_cnt_ip_day_hour     uint16
channel_mean_target         float16
channel_nunique_ip          uint8
click_id                    float64
click_time                  datetime64[ns, Asia/Shanghai]
day                         uint8
device                      uint16
device_mean_target          float16
hour                        uint8
hour_nunique_ip_day         uint8
ip                          uint32
ip_mean_target              float16
is_attributed               float64
next_click_dt               uint16
os                          uint16
ip_nunique_channel          uint16
app_channel_mean_target

___

# Save 



In [48]:
## day8 train_sets
train_df = trn_df[trn_df.day==8]
## day9 valid sets 
val_df = trn_df[trn_df.day==9]


In [49]:
print("train shape: ", train_df.shape)
print("valid shape: ", val_df.shape)
print("test shape : ", test_df.shape)


train shape:  (62360949, 28)
valid shape:  (62832642, 28)
test shape :  (18790469, 28)


In [50]:
fileno = 5
trn_day = 'day8'
val_day = 'day9'
print('../../feat{}_trn_{}_val_{}'.format(fileno,trn_day,val_day))
store = pd.HDFStore('../input/feat/feat{}_trn_{}_val_{}.h5'.format(fileno,trn_day,val_day))
store['train_df'] = train_df
store['valid_df'] = val_df
store['test_df'] = test_df
store.close()

../../feat5_trn_day8_val_day9


load

In [1]:
import pandas as pd

In [2]:
with pd.HDFStore('../input/feat/feat4_trn_day8_val_day9.h5') as store:
    train_df = store['train_df']
    val_df   = store['valid_df']
    test_df  = store['test_df']